# Partie 1 : Collecter les données

In [24]:
# On importe les librairies dont on va avoir besoin
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [25]:
# On définit les headers
headers = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11'})

In [26]:
# On se connecte sur le site que l'on veut scraper, ici le site espagnol Pisos
pisos = "https://www.pisos.com/alquiler/pisos-madrid/"
response = get(pisos, headers=headers) 

In [27]:
# On regarde si le serveur nous a autourisé à accéder au site (il faut que l'on ait le chiffre 200, si on a 403 ou 404 on ne pourra pas accéder au site)
print(response)

<Response [200]>


In [28]:
# On définit nos trois listes que l'on va remplir en scrappant les pages

prices = []
surface = []
location = []

# On crée une boucle qui va chercher l'info sur une page puis une autre, etc..

for page in range(1,40):
    
    # On accède à la page
    pisos_url = "https://www.pisos.com/alquiler/pisos-madrid/" + str(page) + "/"
    r = get(pisos_url, headers=headers)
    
    # On la parse pour pouvoir ensuite la scrapper
    page_html = BeautifulSoup(r.text, 'html.parser')
    
    # On définit un objet qui contient les informations de l'annonce
    house_containers = page_html.find_all('div', class_="information")
    
    if house_containers != []:
        # Dans ces objects (non vides) on va chercher les différentes informations (inspecte le code source de la page pour comprendre pourquoi j'ai mis ces class_ en particulier)
        for container in house_containers:
        
            # Price
            price = container.find('div', class_="price").text
            prices.append(price)
            
            # Surface
            area = container.find('div', class_="item").text
            surface.append(area)
            
            # Location
            zone = container.find('div', class_="location").text
            location.append(zone)

In [29]:
# On crée un DataFrame à partir des données collectées dans nos trois listes

cols = ['Price', 'Size (m²)', 'Zone']

madrid = pd.DataFrame({
    
    'Price': prices,
    'Size (m²)': surface,
    'Zone' : location
                           })[cols]

# On les exporte au format xls et csv

madrid.to_excel('madrid_raw.xls')
madrid.to_csv('madrid_raw.csv')

# 2. Analyse intermédiaire

In [30]:
# On jette un coup d'oeil pour voir qu'il va falloir faire du nettoyage
madrid.head()

,Price,Size (m²),Zone
0,\r\n 2.100 €/mes\n,\r\n 313 m²\r\n ...,\r\n La Berzosilla (Torrelodone...
1,\r\n 700 €/mes\n,\r\n 60 m²\r\n ...,\r\n Zona Noreste (Torrejón de ...
2,\r\n 750 €/mes\n,\r\n 80 m²\r\n ...,\r\n San Agustín del Guadalix\r...
3,\r\n 1.500 €/mes\n,\r\n 255 m²\r\n ...,\r\n San Agustín del Guadalix\r...
4,\r\n 750 €/mes\n,\r\n 63 m²\r\n ...,\r\n Casco Antiguo (Alcorcón)\r...


In [31]:
# On remarque aussi qu'il va falloir changer le type des colonnes pour pouvoir faire des régressions dessus 
madrid.dtypes

Price        object
Size (m²)    object
Zone         object
dtype: object

In [32]:
# On regarde s'il manque des données (pas besoin puisque comme tu as scrappé les données tu sais que le DataFrame est forcéement complet) 
madrid.isna().sum()

Price        0
Size (m²)    0
Zone         0
dtype: int64

# 3. Nettoyage des données

In [33]:
# On utilise les expressions régulières pour nettoyer notre DataFrame 
madrid = madrid.replace(r'\n',' ', regex=True)
madrid = madrid.replace(r'\r',' ', regex=True)
madrid = madrid.replace(r'€/mes',' ', regex=True)
madrid = madrid.replace(r'm²',' ', regex=True)

In [34]:
madrid.head()

,Price,Size (m²),Zone
0,2.100,313 ...,La Berzosilla (Torrelodones)...
1,700,60 ...,Zona Noreste (Torrejón de Ar...
2,750,80 ...,San Agustín del Guadalix ...
3,1.500,255 ...,San Agustín del Guadalix ...
4,750,63 ...,Casco Antiguo (Alcorcón) ...


In [35]:
madrid.dtypes

Price        object
Size (m²)    object
Zone         object
dtype: object

In [36]:
# On va convertir les deux premières colonnes le errors ='coerce' est très important 
# car il transforme les problèmes de conversion en des 'Nan', après cela il suffit de dropna pour obtenir un DataFrame nettoyé

# Price
madrid['Price'] = pd.to_numeric(madrid['Price'], errors ='coerce')
madrid.dropna(subset=['Price'], inplace=True)

# Size (m²)
madrid['Size (m²)'] = pd.to_numeric(madrid['Size (m²)'], errors='coerce')
madrid.dropna(subset=['Size (m²)'], inplace=True)

In [38]:
# Seul petit problème dont je n'ai pas encore la solution, les loyers supérieurs ont été divisé par un facteur mille
madrid.head()

,Price,Size (m²),Zone
0,2.1,313.0,La Berzosilla (Torrelodones)...
1,700.0,60.0,Zona Noreste (Torrejón de Ar...
2,750.0,80.0,San Agustín del Guadalix ...
3,1.5,255.0,San Agustín del Guadalix ...
4,750.0,63.0,Casco Antiguo (Alcorcón) ...


In [39]:
madrid.dtypes

Price        float64
Size (m²)    float64
Zone          object
dtype: object